# DeepSeek Agent 实战：小红书爆款文案生成助手

本 Notebook 将指导您如何使用 DeepSeek LLM 构建一个能够生成小红书爆款文案的智能 Agent。我们将从需求拆解开始，逐步定义 Agent 的系统提示词 (System Prompt)、外部工具 (Tools)，并实现其核心的工作流程，最终生成符合小红书平台特点的文案。

## 1. 环境准备与DeepSeek API配置

In [1]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DASHSCOPE_API_KEY")
if not api_key:
    raise ValueError("请设置 DASHSCOPE_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

dashscope_client = OpenAI(
    api_key=api_key,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)



### 1.1 大模型及Embedding调用函数准备

In [2]:
def ask_llm_with_tools(messages: list[dict], model: str = "deepseek-v3", tools: list[dict] = None):
    """
    使用 LLM 模型，并指定可用的工具。
    """
    response = dashscope_client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools, # 告知模型可用的工具
        tool_choice="auto" # 允许模型自动决定是否使用工具
    )
    return response

def embedding(texts: list[str], model: str = "text-embedding-v4"):
    """
    使用 LLM 模型进行向量化。
    """
    embeddings = dashscope_client.embeddings.create(
        model=model,
        input=texts,
        dimensions=64, # 指定向量维度（仅 text-embedding-v3及 text-embedding-v4支持该参数）
        encoding_format="float"
    )
    return embeddings

### 1.2 产品数据库 DashVector 创建及数据转变

In [3]:
!pip install dashvector

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [4]:
import dashvector, os
if "DASHVECTOR_API_KEY" not in os.environ:
    raise ValueError("DASHVECTOR_API_KEY environment variable not set")
DASHVECTOR_API_KEY = os.environ["DASHVECTOR_API_KEY"]
if "DASHVECTOR_ENDPOINT" not in os.environ:
    raise ValueError("DASHVECTOR_ENDPOINT environment variable not set")
DASHVECTOR_ENDPOINT = os.environ["DASHVECTOR_ENDPOINT"]

def search_dashvector(query_text: str, topk: int=3) -> str:
    vector_client = dashvector.Client(
        api_key=DASHVECTOR_API_KEY,
        endpoint=DASHVECTOR_ENDPOINT
    )
    query_embeddings = embedding([query_text])
    query_vector = query_embeddings.data[0].embedding
    collection = vector_client.get('skincare')
    search_results = collection.query(
        vector=query_vector,
        topk=topk
    )
    print("\n" + "=" * 20 + "RAG检索结果" + "=" * 20 + "\n")
    for doc in search_results:
        print(doc)
    return search_results


### 1.3 Tools (工具定义)

Agent 的“双手”由一系列可调用的工具组成。这些工具扩展了 LLM 的能力，使其能够获取实时信息、查询数据库或执行特定操作。在这里，我们定义了三个核心工具：

*   `search_web`: 用于搜索互联网上的实时信息，如最新趋势、用户评价等。
*   `query_product_database`: 用于查询产品数据库，获取产品的详细卖点和特点。**此工具为模拟**。
*   `generate_emoji`: 用于根据文案内容生成恰当的表情符号。**此工具为模拟**。

In [5]:
!pip install tavily-python

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [6]:
import os,json
from tavily import TavilyClient
if "TAVILY_API_KEY" not in os.environ:
    raise ValueError("TAVILY_API_KEY environment variable is not set.")
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

def search_web(query: str) -> str:
    """
    Search the web for a query and return the first result.
    """
    print("Searching the web for:", query)
    response = tavily_client.search(query, max_results=2)
    print("\n" + "=" * 20 + "Web搜索结果" + "=" * 20 + "\n")
    print(json.dumps(response, indent=4, ensure_ascii=False))
    return response

search_restults = search_web("杏仁酸头皮调理液")

Searching the web for: 杏仁酸头皮调理液

====================Web搜索结果====================

{
    "query": "杏仁酸头皮调理液",
    "follow_up_questions": null,
    "answer": null,
    "images": [],
    "results": [
        {
            "url": "https://www.dr-hsieh.com/blogs/%E6%9C%80%E6%96%B0%E6%B6%88%E6%81%AF/114160?srsltid=AfmBOoqgFV2DgItxxWZ8lKYO2Kk26egW9Ob-jW064pGIxZVpUT5AdHi2",
            "title": "頭皮護理不夠看！獻上新頭皮保養「杏仁酸控油調理洗髮精 ... - Dr.Hsieh",
            "content": "使用含有高濃度杏仁酸精華，有效深層滲透頭皮毛孔，輔助頭皮清潔和幫助頭部去角質的作用，另外抗菌功效，還能通暢頭皮毛孔阻塞，讓塞住的頭皮毛囊炎恢復健康。",
            "score": 0.56258565,
            "raw_content": null
        },
        {
            "url": "https://shopee.tw/%E2%9D%A4%EF%B8%8Farin-%E6%B0%A7%E6%BD%A4-%E9%85%B7%E6%A8%82%E9%A0%AD%E7%9A%AE%E6%AD%A2%E7%99%A2%E6%B6%B2-%E9%AB%98%E6%95%88%E6%8E%A7%E6%B2%B9-%E9%A0%AD%E7%9A%AE%E8%AA%BF%E7%90%86%E6%B6%B210-%E6%9D%8F%E4%BB%81%E9%85%B8%E7%85%A5%E8%86%9A%E7%B2%BE%E8%8F%AF-30ml-i.17068797.4240679022",
            "title": "arin 氧潤酷樂頭皮止癢液高效控油頭皮調理液10%杏仁酸煥膚精華 

In [7]:
def query_product_database(product_name: str) -> str:
    """查询产品数据库，返回产品信息。"""
    ret = search_dashvector(product_name)
    product_info = ret[0].fields["text"]
    return product_info

query_product_database("杏仁酸头皮调理液")


====================RAG检索结果====================

{"id": "49", "fields": {"name": "杏仁酸头皮调理液", "description": "1.5%杏仁酸调节毛囊口角质，减少头屑生成并蓬松发根", "text": "名称：杏仁酸头皮调理液，描述：1.5%杏仁酸调节毛囊口角质，减少头屑生成并蓬松发根，类别：头皮护理", "category": "头皮护理"}, "score": 0.0}
{"id": "28", "fields": {"name": "艾地苯抗氧精华", "description": "0.5%艾地苯醌醌复合麦角硫因，中和多种自由基伤害，防护环境污染损伤", "text": "名称：艾地苯抗氧精华，描述：0.5%艾地苯醌醌复合麦角硫因，中和多种自由基伤害，防护环境污染损伤，类别：精华", "category": "精华"}, "score": 0.3049}
{"id": "34", "fields": {"name": "玻尿酸导入面膜贴", "description": "双分子玻尿酸梯度渗透，微压膜布促进活性物传输，即时改善缺水细纹", "text": "名称：玻尿酸导入面膜贴，描述：双分子玻尿酸梯度渗透，微压膜布促进活性物传输，即时改善缺水细纹，类别：面膜", "category": "面膜"}, "score": 0.3157}


'名称：杏仁酸头皮调理液，描述：1.5%杏仁酸调节毛囊口角质，减少头屑生成并蓬松发根，类别：头皮护理'

In [8]:
def generate_emoji(context: str):
    sys_message = """
    你是一个emoji生成器，你需要根据输入的文本生成5个emoji符号，并返回一个以逗号分隔的emoji符号列表。
    格式如下：["🌟", "😲", "💖", "✨", "👍"]
    """
    messages = [
        {"role": "system", "content": sys_message},
        {"role": "user", "content": context}
    ]
    response = ask_llm_with_tools(messages)
    return response.choices[0].message.content


## 2. 小红书内容生成system message, user message以及可用工具定义

### 2.1 System message准备

In [9]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "💧深海蓝藻保湿面膜｜干皮救星！敷完水嫩到发光✨",
  "body": "姐妹们！发现了一款神仙面膜！就是这个【深海蓝藻保湿面膜】！😍\n\n最近换季皮肤干到爆炸，连粉底都卡纹...偶然用了这款面膜，真的被惊艳到了！敷上去凉凉的超级舒服，精华液超级多但一点都不黏腻～\n\n✨主打深海蓝藻提取物，补水锁水能力绝了！敷完脸就像喝饱水一样，嫩到反光！而且超级温和，敏感肌也能用～\n\n我一般一周敷2-3次，现在皮肤状态超级稳定，上妆都服帖多了！绝对是干皮/混干皮的年度爱用！\n\nPS：冬天可以先用温水泡一下再敷，更舒服哦～",
  "hashtags": ["#保湿面膜", "#干皮救星", "#深海蓝藻", "#换季护肤", "#敏感肌友好"],
  "emojis": ["💧", "😍", "✨", "🌿", "🎁"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

### 2.2 工具定义

In [10]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

In [11]:
# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": search_web,
    "query_product_database": query_product_database,
    "generate_emoji": generate_emoji,
}

## 3. 构建小红书文案生成 Agent

现在，我们将把 System Prompt、工具定义和模拟工具函数整合起来，构建出能够自动执行的 DeepSeek Agent 工作流。核心是 `generate_rednote` 函数，它通过一个循环来模拟 Agent 的 `Thought-Action-Observation` 过程。

In [12]:
import json
import re


def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。

    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。

    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """

    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")

    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]

    iteration_count = 0
    final_response = None

    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")
        
        try:
            # 调用 DeepSeek API，传入对话历史和工具定义
            response = ask_llm_with_tools(messages, model="deepseek-v3", tools=TOOLS_DEFINITION)
            response_message = response.choices[0].message
            
            # **ReAct模式：处理工具调用**
            if response_message.tool_calls: # 如果模型决定调用工具
                print("Agent: 决定调用工具...")
                messages.append(response_message) # 将工具调用信息添加到对话历史
                
                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    # 确保参数是合法的JSON字符串，即使工具不要求参数，也需要传递空字典
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}

                    print(f"Agent Action: 调用工具 '{function_name}'，参数：{function_args}")
                    
                    # 查找并执行对应的模拟工具函数
                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: 工具返回结果：{tool_result}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result) # 工具结果作为字符串返回
                        })
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_message
                        })
                messages.extend(tool_outputs) # 将工具执行结果作为 Observation 添加到对话历史
                
            # **ReAct 模式：处理最终内容**
            elif response_message.content: # 如果模型直接返回内容（通常是最终答案）
                print(f"[模型生成结果] {response_message.content}")
                
                # --- START: 添加 JSON 提取和解析逻辑 ---
                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)
                
                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 任务完成，成功解析最终JSON文案。")
                        # return json.dumps(final_response, ensure_ascii=False, indent=2)
                        return json.dumps(final_response, ensure_ascii=False, indent=2, separators=(',', ': '))
                    except json.JSONDecodeError as e:
                        print(f"Agent: 提取到JSON块但解析失败: {e}")
                        print(f"尝试解析的字符串:\n{extracted_json_content}")
                        messages.append(response_message) # 解析失败，继续对话
                else:
                    # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 任务完成，直接解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                        messages.append(response_message) # 非JSON格式，继续对话
                # --- END: 添加 JSON 提取和解析逻辑 ---
            else:
                print("Agent: 未知响应，可能需要更多交互。")
                break
                
        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break
    
    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

## 4. 实际测试与文案生成

现在，让我们调用我们构建的 `generate_rednote` 函数，看看它能生成什么样的爆款文案！

In [20]:
# 测试案例 1: 深海蓝藻保湿面膜
product_name_1 = "深海蓝藻保湿面膜"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：深海蓝藻保湿面膜，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '深海蓝藻保湿面膜'}

====================RAG检索结果====================

{"id": "5", "fields": {"name": "深海微藻防晒乳", "description": "SPF50+ PA++++全波段防护体系，复配深海藻类提取物中和光老化损伤，轻薄质地不堵塞毛孔", "text": "名称：深海微藻防晒乳，描述：SPF50+ PA++++全波段防护体系，复配深海藻类提取物中和光老化损伤，轻薄质地不堵塞毛孔，类别：防晒", "category": "防晒"}, "score": 0.2367}
{"id": "21", "fields": {"name": "酵母发酵修护原液", "description": "清酒酒粕发酵滤液含丰富小分子肽，调节角质代谢节奏，重建健康表皮微生态", "text": "名称：酵母发酵修护原液，描述：清酒酒粕发酵滤液含丰富小分子肽，调节角质代谢节奏，重建健康表皮微生态，类别：精华", "category": "精华"}, "score": 0.319}
{"id": "4", "fields": {"name": "烟酰胺净透美白精华", "description": "5%烟酰胺浓度配合AA2G衍生物，抑制黑色素小体转移，改善肤色不均及痘印沉淀问题", "text": "名称：烟酰胺净透美白精华，描述：5%烟酰胺浓度配合AA2G衍生物，抑制黑色素小体转移，改善肤色不均及痘印沉淀问题，类别：精华", "category": "精华"}, "score": 0.3203}
Observation: 工具返回结果：名称：深海微藻防晒乳，描述：SPF50+ PA++++全波段防护体系，复配深海藻类提取物中和光老化损伤，轻薄质地不堵塞毛孔，类别：防晒
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'search_web'，参数：{'query': 

In [14]:
# 测试案例 2: 美白精华
product_name_2 = "美白精华"
tone_style_2 = "知性温柔"
result_2 = generate_rednote(product_name_2, tone_style_2)

print("\n--- 生成的文案 2 ---")
print(result_2)


🚀 启动小红书文案生成助手，产品：美白精华，风格：知性温柔

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '美白精华'}

====================RAG检索结果====================

{"id": "4", "fields": {"name": "烟酰胺净透美白精华", "description": "5%烟酰胺浓度配合AA2G衍生物，抑制黑色素小体转移，改善肤色不均及痘印沉淀问题", "text": "名称：烟酰胺净透美白精华，描述：5%烟酰胺浓度配合AA2G衍生物，抑制黑色素小体转移，改善肤色不均及痘印沉淀问题，类别：精华", "category": "精华"}, "score": 0.2208}
{"id": "10", "fields": {"name": "视黄醇夜间抗皱精华", "description": "0.3%包裹缓释视黄醇配合肽复合物，激活胶原再生通路，改善静态皱纹及皮肤粗糙度", "text": "名称：视黄醇夜间抗皱精华，描述：0.3%包裹缓释视黄醇配合肽复合物，激活胶原再生通路，改善静态皱纹及皮肤粗糙度，类别：精华", "category": "精华"}, "score": 0.2465}
{"id": "31", "fields": {"name": "依克多因防护精华", "description": "盐生微生物提取依克多因，稳定蛋白质结构，对抗极端环境损伤", "text": "名称：依克多因防护精华，描述：盐生微生物提取依克多因，稳定蛋白质结构，对抗极端环境损伤，类别：精华", "category": "精华"}, "score": 0.3085}
Observation: 工具返回结果：名称：烟酰胺净透美白精华，描述：5%烟酰胺浓度配合AA2G衍生物，抑制黑色素小体转移，改善肤色不均及痘印沉淀问题，类别：精华
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '美白精华效果惊艳'}
Observation: 工具返回

In [15]:
# 测试案例 3: 祛痘喷雾
product_name_3 = "祛痘喷雾"
tone_style_3 = "惊喜推荐"
result_3 = generate_rednote(product_name_2, tone_style_2)

print("\n--- 生成的文案 3 ---")
print(result_3)


🚀 启动小红书文案生成助手，产品：美白精华，风格：知性温柔

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '美白精华'}

====================RAG检索结果====================

{"id": "4", "fields": {"name": "烟酰胺净透美白精华", "description": "5%烟酰胺浓度配合AA2G衍生物，抑制黑色素小体转移，改善肤色不均及痘印沉淀问题", "text": "名称：烟酰胺净透美白精华，描述：5%烟酰胺浓度配合AA2G衍生物，抑制黑色素小体转移，改善肤色不均及痘印沉淀问题，类别：精华", "category": "精华"}, "score": 0.2208}
{"id": "10", "fields": {"name": "视黄醇夜间抗皱精华", "description": "0.3%包裹缓释视黄醇配合肽复合物，激活胶原再生通路，改善静态皱纹及皮肤粗糙度", "text": "名称：视黄醇夜间抗皱精华，描述：0.3%包裹缓释视黄醇配合肽复合物，激活胶原再生通路，改善静态皱纹及皮肤粗糙度，类别：精华", "category": "精华"}, "score": 0.2465}
{"id": "31", "fields": {"name": "依克多因防护精华", "description": "盐生微生物提取依克多因，稳定蛋白质结构，对抗极端环境损伤", "text": "名称：依克多因防护精华，描述：盐生微生物提取依克多因，稳定蛋白质结构，对抗极端环境损伤，类别：精华", "category": "精华"}, "score": 0.3085}
Observation: 工具返回结果：名称：烟酰胺净透美白精华，描述：5%烟酰胺浓度配合AA2G衍生物，抑制黑色素小体转移，改善肤色不均及痘印沉淀问题，类别：精华
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '美白精华 知性温柔'}
Observation: 工具返

## 5. 格式化 小红书文案

**格式化函数 `format_rednote_for_markdown` 的功能：**

1. 解析 JSON 字符串。
2. 提取标题、正文、标签和表情符号。
3. 将它们组合成一个易读的 Markdown 格式的文本。


**工作方式：**

1. **解析 JSON**：使用 `json.loads()` 将输入的字符串转换为 Python 字典。如果解析失败，会返回一个错误信息。
2. **提取数据**：使用 `.get()` 方法从字典中安全地提取 `title`、`body` 和 `hashtags`。使用 `.get()` 的好处是，如果某个键不存在，它会返回一个默认值（例如 `None` 或空列表），而不是抛出 `KeyError`。
3. **构建 Markdown 标题**：将 `title` 格式化为 Markdown 的二级标题 (`## Title`)。
4. **处理正文**：直接使用 `body`。由于小红书正文中的换行很重要，我们保留它们。
5. **处理 Hashtags**：将 `hashtags` 列表中的每个标签用空格连接起来，形成一行。
6. **表情符号 (Emojis)**：在小红书的实际发布中，表情符号通常已经嵌入在标题和正文中了。这个函数没有单独列出它们，因为这通常不是最终发布格式的一部分。如果需要，可以取消注释相关代码来单独显示它们。
7. **返回结果**：返回拼接好的 Markdown 字符串，并使用 `.strip()` 去除可能存在于末尾的多余空白。

In [16]:
import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    if emojis:
        emoji_string = " ".join(emojis)
        markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [21]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_1)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## 💦深海蓝藻保湿面膜｜干敏肌的补水救星！敷完脸蛋嫩到发光✨

姐妹们！我又挖到宝啦！就是这个【深海蓝藻保湿面膜】！😍

最近换季皮肤干到爆炸，上妆卡粉到怀疑人生...偶然用了这款面膜，真的被惊艳到了！敷上去凉凉的超舒服，精华液多到可以涂全身但一点都不黏腻～

✨主打深海蓝藻提取物，补水锁水能力绝绝子！敷完脸就像喝饱水一样，嫩到反光！而且超级温和，我这个敏感肌用着也毫无压力～

我一般一周敷2-3次，现在皮肤状态超级稳定，上妆都服帖多了！绝对是干皮/混干皮的年度爱用！

PS：冬天可以先用温水泡一下再敷，更舒服哦～

#保湿面膜 #干敏肌救星 #深海蓝藻 #换季护肤 #敏感肌友好

使用的表情：💦 😍 ✨ 🌿 🎁


In [18]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_2)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## ✨烟酰胺净透美白精华｜告别暗沉，28天白到发光🌟

姐妹们！最近发现了一款让我惊艳的美白精华！就是这个【烟酰胺净透美白精华】！😍

一直因为熬夜和压力，皮肤暗沉到不行，痘印也消不掉...用了这款精华后，真的被它的效果折服了！质地轻薄好吸收，完全不会黏腻～

✨主打5%烟酰胺浓度配合AA2G衍生物，不仅能抑制黑色素转移，还能改善肤色不均和痘印！用了28天，皮肤真的透亮了好多，连朋友都问我是不是打了光💖

我一般是早晚各用一次，后续搭配防晒，效果更明显！敏感肌也可以放心用，温和不刺激～

PS：记得坚持用哦，美白是个持久战，但这款精华真的值得！👍

#美白精华 #烟酰胺 #肤色提亮 #痘印修复 #敏感肌友好

使用的表情：✨ 🌟 😍 💖 👍


In [19]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_3)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## ✨烟酰胺美白精华｜告别暗沉！28天见证肌肤透亮之旅🌸

亲爱的姐妹们，今天要分享的是这款让我惊艳的【烟酰胺净透美白精华】！💖

作为一个长期与暗沉肌肤抗争的人，这款精华真的是我的救星！含有5%高浓度烟酰胺，搭配AA2G衍生物，不仅能有效抑制黑色素转移，还能改善肤色不均和痘印沉淀～连续使用28天后，我的肌肤明显透亮了许多，连朋友都问我是不是换了粉底液！

💧质地轻盈不黏腻，吸收超级快，早晚都能用！我习惯在爽肤水后使用，轻轻按摩至吸收，再配合防晒，效果加倍哦～

🌙特别推荐给和我一样有暗沉、痘印困扰的姐妹！温和不刺激，敏感肌也可安心使用～现在我的肌肤终于告别了灰蒙蒙的状态，素颜也敢出门啦！

#美白精华 #烟酰胺 #28天美白计划 #告别暗沉 #敏感肌友好

使用的表情：✨ 🌸 💧 🌙 💖
